# An AS/AD-model for a closed economy in the short run

**This project analyzes the effects of shocks to a simple AS/AD-model for a closed economy in the short run. We use the model to analyze supply- and demand shocks as well as minimizing a social loss function.**

Imports and set magics:

In [34]:
import sympy as sm
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, IntSlider, FloatSlider
import scipy.optimize as opt

plt.rcParams.update({"axes.grid":True,"grid.color":"darkblue","grid.alpha":"0.3","grid.linestyle":"--"})
plt.rcParams.update({'font.size': 12})

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

import modelproject
from modelproject import ASAD



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of modelproject failed: Traceback (most recent call last):
  File "/Users/laugekoch-klarskov/opt/anaconda3/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/Users/laugekoch-klarskov/opt/anaconda3/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 455, in superreload
    module = reload(module)
  File "/Users/laugekoch-klarskov/opt/anaconda3/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/Users/laugekoch-klarskov/Desktop/uni/KA - 1. semester/projects-2024-william-ellen-og-lauge/modelproject/modelproject.py", line 142, in <module>
    model.plot_social_loss()
NameError: name 'model' is not defined
]


# Model description

The model used in this project is a version of a closed economy AS/AD-model as outlined in "Introducing Advanced Macroeconomics - Growth and Business Cycles" by Peter Birch Sørensen & Hans Jørgen Whitta-Jacobsen.  

We use the following equations to describe our model: 

**1: IS-curve** - Market equilibrium with government spending and taxes.

$$
y_t-\bar{y} = \alpha_1 (g_t-\bar{g}) - \alpha_2(r_t-\bar{r})-\alpha_{3}(\tau_{t}-\overline{\tau})
$$

**2: Fischer Equation** - In ex-ante (modeled by expected inflation).

$$
r_t = i_t-\pi^{e}_{t+1}
$$

**3: Monetary policy rule** - The Taylor-Rule with stabilization wrt. both inflation and output

$$
i_t = \bar{r} + \pi^{e}_{t+1} + h(\pi_t - \pi^*) + b(y_t- \bar{y})
$$

**4: The SRAS curve** - This is derived from the expectations-augmented Phillips-curve. For given inflation expectations, there is a positive correlation between inflation and outputgap. 

$$
\pi_t = \pi_t^e + \gamma (y_t-\bar{y}) + s_t
$$


**5: Inflation expectations** - This equation shows that the agents have static inflation expectations, since they expect inflation in period t to be equal to observed inflation in the previous period. 

$$
\pi_t^e = \pi_{t-1}
$$

From the above equations, we can define the AD-curve and the AS-curve: 


The AD-curve combines equations 1, 2 and 3. Inserting 3 in 2:

$$
r_t = \bar{r} + \pi^{e}_{t+1} + h(\pi_t - \pi^*) + b(y_t- \bar{y})-\pi^{e}_{t+1}
$$
$$
r_t = \bar{r} + h(\pi_t - \pi^*) + b(y_t- \bar{y})
$$
Inserting in eq. 1:
$$
y_t-\bar{y} = \alpha_1 (g_t-\bar{g}) - \alpha_2(\bar{r} + h(\pi_t - \pi^*) + b(y_t- \bar{y})-\bar{r})-\alpha_{3}(\tau_{t}-\overline{\tau})
$$
$$
y_t-\bar{y} = \alpha_1 (g_t-\bar{g}) - \alpha_2(h(\pi_t - \pi^*) + b(y_t- \bar{y}))-\alpha_{3}(\tau_{t}-\overline{\tau})
$$
Isolating $y_t-\bar{y}$:
$$
y_t-\bar{y} = z-\alpha(\pi_t - \pi^*)
$$
where
$$
z = \frac{\alpha_1}{1+\alpha_2b}(g_t-\bar{g})-\frac{\alpha_3}{1+\alpha_2b}(\tau_t-\bar{\tau}), \alpha = \frac{\alpha_2 h}{1+\alpha_2b}
$$

The AS-curve is found by inserting equation 5 in 4:

$$
\pi_t = \pi_{t-1} + \gamma(y_t-\bar{y}) + s_t $$

## Analytical solution

We firstly define our model in python using a class in our py-file. We use the model to make plots with interactive sliders for the demand shock, z, and the supply shock, s. 



In [35]:
def interactive_gap(T, z, s, z_duration, s_duration):
    model = ASAD(T=T, z=z, s=s, z_duration=z_duration, s_duration=s_duration)
    model.solve_model()
    model.plot_results()

# Create sliders for interactive input
T_slider = IntSlider(min=10, max=100, step=1, value=20, description='Total Periods (T)')
z_slider = FloatSlider(min=-0.2, max=0.2, step=0.01, value=0.01, description='Demand Shock (z)')
s_slider = FloatSlider(min=-0.2, max=0.2, step=0.01, value=0, description='Supply Shock (s)')
z_duration_slider = IntSlider(min=0, max=50, step=1, value=1, description='Duration of z')
s_duration_slider = IntSlider(min=0, max=50, step=1, value=0, description='Duration of s')

# Display the interactive plot
interactive_plot = interactive(interactive_gap, T=T_slider, z=z_slider, s=s_slider, z_duration=z_duration_slider, s_duration=s_duration_slider)
display(interactive_plot)

interactive(children=(IntSlider(value=20, description='Total Periods (T)', min=10), FloatSlider(value=0.01, de…

**Temporary demand shock:**
In Figure 1, the initial values show a demand shock, z, of value 0.01, with a duration of 1 period (and no supply shock). This shock could arise from increased government spending, reduced taxes, or adjustments in monetary policy parameters such as the sensitivity of nominal interest rates to output gap changes. Initially, both the output gap and inflation rise in response to this positive shock. In the subsequent period, although the demand shock dissipates, static inflation expectations persist among economic agents, leading them to anticipate continued higher inflation. However, actual inflation turns out to be lower than expected, resulting in a negative output gap. As agents gradually revise their expectations to match observed inflation, inflation converges back to its original level post-shock.

**Temporary supply shock:**
If we instead change the sliders to a negative value of $s=-0.01$ and a duration of s of 1 (and no demand shock), this corresponds to a positive supply shock (e.g., more production at lower costs). We find that the output gap increases and after the first period converges back to zero, while the inflation gap becomes negative and gradually converges back to zero. We will elaborate on the effects in the AS-AD-plot below. 

In [36]:
def interactive_model(T, z, s, gamma, z_duration, s_duration):
    model = ASAD(T=T, gamma=gamma, z=z, s=s, z_duration=z_duration, s_duration=s_duration)
    model.solve_model()
    model.plot_ad_as()

# Create sliders for interactive input
T_slider = IntSlider(min=1, max=50, step=1, value=20, description='Total Periods')
z_slider = FloatSlider(min=-0.1, max=0.1, step=0.01, value=0.01, description='Demand Shock (z)')
s_slider = FloatSlider(min=-0.1, max=0.1, step=0.01, value=0, description='Supply Shock (s)')
gamma_slider = FloatSlider(min=0.1, max=2.0, step=0.1, value=1.0, description='Gamma')
z_duration_slider = IntSlider(min=0, max=50, step=1, value=1, description='Duration of z')
s_duration_slider = IntSlider(min=0, max=50, step=1, value=0, description='Duration of s')

# Display interactive widgets
interactive_plot = interactive(interactive_model, T=T_slider, z=z_slider, s=s_slider, gamma=gamma_slider, z_duration=z_duration_slider, s_duration=s_duration_slider)
display(interactive_plot)

interactive(children=(IntSlider(value=20, description='Total Periods', max=50, min=1), FloatSlider(value=0.01,…

Figure 2 shows the AS-AD-model and includes sliders for the shocks, as in Figure 1. 

**Temporary demand shock:**
The initial values show a demand shock, z, of value 0.01, with a duration of 1 period (and no supply shock). We see that in period 1, the demand shock raises output and inflation above long-term levels (economic boom). Increased demand boosts production and employment, leading to higher costs and inflation. The central bank responds with a tightening policy more than one-for-one with inflation, raising real interest rates and dampening demand. In period 2, the AD-curve shifts back to the initial AD-curve, as the demand shock fades. Higher inflation expectations from period 1 shift AS left/up in period 2, anticipating inflation. This drives wage demands, accelerates prices, and sustains elevated inflation. The central bank continues tightening to stabilize inflation. In the subsequent periods, inflation expectations adjust to period the former periods’ observed inflation, shifting AS downward. Over time, output returns to its long-term level and inflation stabilizes. Nominal wage rigidities and slow expectation adjustments influence this process.

**Temporary supply shock:**
If we instead change the sliders to a negative value of $s=-0.01$ and a duration of s of 1 (and no demand shock), this corresponds to a positive supply shock, like in the analysis for Figure 1 above. In the first period, a temporary positive supply shock ($s < 0$) shifts AS down by the size of the shock s, creating a new short-term equilibrium. Inflation falls to $\pi_1 < \pi^*$, but not entirely with the shock’s size, as the central bank responds by easing monetary policy (lowering interest rates), boosting output ($y_1>\bar{y}$). In the second period where $s=0$, AS shifts up. However, static expectations prompt the agents to adjust inflation expectations, keeping the AS-curve from returning directly. In all subsequent periods, inflation expectations adjust to observed inflation in the former period, pushing AS gradually up. Successive expectation adjustments lead output towards $\bar{y}$ and inflation towards $\pi^*$. Persistence in convergence is due to nominal wage rigidities and slow adjustment of expectations.

In [37]:
def interactive_perm(T, z, s, gamma):
    z_duration = s_duration = T  # Set both durations equal to T for permanent shocks
    model = ASAD(T=T, gamma=gamma, z=z, s=s, z_duration=z_duration, s_duration=s_duration)
    model.solve_model()
    model.plot_ad_as()

# Create sliders for interactive input
T_slider = IntSlider(min=1, max=50, step=1, value=20, description='Total Periods (T)')
z_slider = FloatSlider(min=-0.1, max=0.1, step=0.001, value=0.01, description='Demand Shock (z)', readout_format='.3f')
s_slider = FloatSlider(min=-0.01, max=0.01, step=0.001, value=0, description='Supply Shock (s)', readout_format='.3f')
gamma_slider = FloatSlider(min=0.1, max=2.0, step=0.1, value=1.0, description='Gamma')

# Display the interactive plot
interactive_plot = interactive(interactive_perm, T=T_slider, z=z_slider, s=s_slider, gamma=gamma_slider)
display(interactive_plot)



interactive(children=(IntSlider(value=20, description='Total Periods (T)', max=50, min=1), FloatSlider(value=0…

In Figure 3, the durations of the shocks are set equal to T (total periods), so changing the slider for the shocks shows the effects of permanent shocks in the AS-AD-model. 

**Permanent demand shocks:**
We find that demand shocks do not affect the economy in the long run. In the long run, a demand shock in an AS-AD economy leads to the economy returning to its natural output level, $\bar{y}$. Initially higher output and inflation caused by increased demand gradually stabilize as prices, wages, and inflation expectations adjust back to their long-term equilibrium levels. Central banks typically adjust monetary policy to ensure inflation returns to its target, contributing to the economy's eventual return to equilibrium.

**Permanent supply shocks:**
Supply shocks, however, affect both the AS and AD-curves in the long run. When there is a positive supply shock, such as technological advancements or increased productivity, the AS curve shifts to the right, reflecting higher potential output levels and lower costs for producers. This shift leads to:
* Gradual decreases in prices as goods become more abundant and cheaper to produce.
* Initially, an increase in real incomes and purchasing power due to lower prices.
* Over time, the AD curve shifts rightward as consumers and businesses respond to lower prices by increasing their spending and investment.
Conversely, a negative supply shock, like higher production costs or reduced productivity, shifts the AS curve to the left:
* Output decreases as it becomes more costly to produce goods and services.
* Prices rise as supply becomes constrained, reducing consumer purchasing power.
* Initially, the AD curve may shift leftward as higher prices reduce consumer spending and investment.


## Social loss function 

In this section, we look at the social loss of shocks to inflation and output. Social loss is minimized when the following function is minimized:
$$ L_t = (\pi_t - \pi^*)^2 + \alpha (y_t - \bar{y})^2 $$


In [38]:
def interactive_SL(T, z, s, z_duration, s_duration):
    model = ASAD(T=T, z=z, s=s, z_duration=z_duration, s_duration=s_duration)
    model.solve_model()
    #model.plot_results()
    model.plot_social_loss()

# Create sliders for interactive input
T_slider = IntSlider(min=10, max=100, step=1, value=20, description='Total Periods (T)')
z_slider = FloatSlider(min=-0.2, max=0.2, step=0.01, value=0.01, description='Demand Shock (z)')
s_slider = FloatSlider(min=-0.2, max=0.2, step=0.01, value=0, description='Supply Shock (s)')
z_duration_slider = IntSlider(min=0, max=50, step=1, value=1, description='Duration of z')
s_duration_slider = IntSlider(min=0, max=50, step=1, value=0, description='Duration of s')

# Display the interactive plot
interactive_plot = interactive(interactive_SL, T=T_slider, z=z_slider, s=s_slider, z_duration=z_duration_slider, s_duration=s_duration_slider)
display(interactive_plot)

interactive(children=(IntSlider(value=20, description='Total Periods (T)', min=10), FloatSlider(value=0.01, de…

Figure 4 illustrates the social loss function resulting from the different shocks from adjusting the sliders. Initially, both shocks lead to increased social costs, peaking during the shock periods. Over time, social loss gradually diminishes and approaches zero.
This trend highlights that while initial shocks disrupt economic stability, the economy adapts and mitigates these impacts over the longer term. Ultimately, the convergence to zero reflects the economy's ability to return to equilibrium despite temporary disturbances.

## NUMERICAL SOLUTION   

In Hans Jørgen Whitta-Jacobsen and Peter Birch Sørensen’s book, "Introduction to Advanced Macroeconomics," we the optimal parameter values $\alpha = 0.7$ and $\gamma = 0.075$, which we utilize for the numerical solution.

## OBS!!!!

In [39]:
#Evaluating the social loss at the optimal parameter values
optimal_g = optimal_par_gh.x[0]
optimal_alpha_1 = optimal_par_gh.x[1]
optimal_social_loss = social_loss([optimal_g, optimal_alpha_1])

#Generating a range of parameter combinations
g_values = np.linspace(0, 0.1, 100)   #Generating 100 equally spaced values between 0 and 0.1 for parameter g
alpha_1_values = np.linspace(0, 3, 100)   #Generating 100 equally spaced values between 0 and 3 for parameter alpha_1
social_loss_values = np.zeros((100, 100))   #Creating a 100x100 array to store social loss values

#Calculating social loss for each parameter combination
for i, g in enumerate(g_values):
    for j, alpha_1 in enumerate(alpha_1_values):
        social_loss_values[i, j] = social_loss([g, alpha_1])   #Calculating social loss for each parameter combination

#Finding the minimum social loss and its corresponding parameter values
min_loss = np.min(social_loss_values)   #Finding the minimum social loss value from the array
min_loss_idx = np.unravel_index(np.argmin(social_loss_values), social_loss_values.shape)   #Finding the indices of the minimum loss value
min_loss_g = g_values[min_loss_idx[0]]   #Extracting the corresponding value of g for the minimum loss
min_loss_alpha_1 = alpha_1_values[min_loss_idx[1]]   #Extracting the corresponding value of alpha_1 for the minimum loss

#Compare the optimal social loss with the minimum social loss
if np.isclose(optimal_social_loss, min_loss):
    print("The optimal parameter values do indeed minimize the social loss function.")   #Text indicating that the optimal parameter values minimize the social loss function
else:
    print("The optimal parameter values may not minimize the social loss function.")   #Text indicating that they may not minimize the social loss function

#Printing the optimal social loss and the minimum social loss
print("Optimal Social Loss:", optimal_social_loss)   #Print the value of the optimal social loss
print("Minimum Social Loss:", min_loss)   #Print the value of the minimum social loss

NameError: name 'optimal_par_gh' is not defined

Next, we check the convergence of social loss:

In [ ]:
#Defining an empty list to store the social loss values across iterations
social_loss_history = []

#Optimizing with convergence monitoring
def optimization_callback(xk, _):

    #social_loss_val is calculated at each iteration and appendeds it to the social_loss_history list
    social_loss_val = social_loss(xk)
    social_loss_history.append(social_loss_val)
    print("Current social loss:", social_loss_val)

#Optimization using Trust Region Constraint method
#optimal_par_gh minimizes the social_loss function with respect to the parameters [initial_g, initial_alpha_1] 
optimal_par_gh = opt.minimize(social_loss, [initial_g, initial_alpha_1], method='trust-constr', callback=optimization_callback)

#Checking if the social loss has converged by comparing the last two values in social_loss_history
if len(social_loss_history) > 1 and np.isclose(social_loss_history[-1], social_loss_history[-2]):
    print("The optimization has converged.")
else:
    print("The optimization may not have converged.")

#Plotting the convergence of the social loss function
plt.plot(range(len(social_loss_history)), social_loss_history)
plt.xlabel("Iteration")
plt.ylabel("Social Loss")
plt.title("Figure 3: Convergence of Social Loss")
plt.show()

We now check how steady state is affected by monetary policy interventions:

In [ ]:
# Create an instance of monetary_policy_intervention class


# Set the desired interest rate for the policy intervention (decrease in interest rate)
interest_rate = -0.2

# Analyze the policy intervention
steady_state_before, steady_state_after = model.analyze_policy_intervention(interest_rate)

# Compare the steady state values
print("Steady state value before policy intervention:", steady_state_before)
print("Steady state value after policy intervention:", steady_state_after)

## Conclusion

In conclusion, this project has examined the impact of shocks on a basic AS/AD model within a closed economy in the short term. Through our analysis, we investigated the repercussions of variations in shocks and a social loss function.